<a href="https://colab.research.google.com/github/leonardoub/SCRIPT_PALERMO/blob/master/Network_classification_histology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [2]:
#load data from Drive
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
train_dataset_path = '/gdrive/My Drive/AIM_PA/database_training2.csv'
test_dataset_path = '/gdrive/My Drive/AIM_PA/database_nostro_without_nan.csv'

In [0]:
df_train = pd.read_csv(train_dataset_path)
df_test = pd.read_csv(test_dataset_path)

In [0]:
df_train.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_test.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_train.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)
df_test.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)

In [0]:
train_data = df_train.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
test_data = df_test.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
train_labels = df_train.Histology

In [0]:
test_labels = df_test.Histology

##Z score dei dati

In [0]:
mean = train_data.mean(axis=0)
train_data_stand = train_data - mean
std = train_data.std(axis=0)
train_data_stand /= std

In [0]:
test_data_stand = test_data - mean
test_data_stand /= std

##Vettorizzare i label

In [0]:
word_index={'adenocarcinoma':0, 'large cell':1, 'squamous cell carcinoma':2}

In [0]:
train_labels_dec = [word_index[label] for label in train_labels]

In [0]:
test_labels_dec = [word_index[label] for label in test_labels]

In [17]:
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [0]:
one_hot_train_labels = to_categorical(train_labels_dec)
one_hot_test_labels = to_categorical(test_labels_dec)

#PCA

In [0]:
from sklearn.decomposition import PCA

In [0]:
pca = PCA(n_components=0.95, svd_solver='full')

In [27]:
pca.fit(train_data_stand)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

In [0]:
train_data_stand_pca = pca.transform(train_data_stand)
test_data_stand_pca = pca.transform(test_data_stand)

In [59]:
type(train_data_stand_pca)

numpy.ndarray

#Building Network

In [0]:
from keras import models

In [0]:
from keras import layers

In [0]:
def build_model():
  model = models.Sequential()
  model.add(layers.Dense(8, activation='relu', input_shape=(14,)))
  model.add(layers.Dense(6, activation='relu'))
  model.add(layers.Dense(3, activation='softmax'))
  model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

#Stratified k-fold

This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.

In [0]:
from sklearn.model_selection import StratifiedKFold

In [111]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
skf.get_n_splits(train_data_stand_pca, train_labels_dec)

3

In [112]:
for train_index, test_index in skf.split(train_data_stand_pca, train_labels_dec):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [  0   1   4   5   8   9  11  12  14  15  16  17  19  20  22  23  24  25
  27  29  30  33  34  36  37  38  39  40  41  42  44  45  46  48  51  52
  53  56  57  58  59  60  62  63  65  66  67  69  72  76  77  78  79  80
  81  83  84  85  87  88  89  90  92  96  97  98 100 101 102 103 104 105
 107 109 110 111 113 115 117 120 121 122 124 125 127 128] TEST: [  2   3   6   7  10  13  18  21  26  28  31  32  35  43  47  49  50  54
  55  61  64  68  70  71  73  74  75  82  86  91  93  94  95  99 106 108
 112 114 116 118 119 123 126 129 130]
TRAIN: [  2   3   5   6   7   8   9  10  11  12  13  18  20  21  25  26  27  28
  29  30  31  32  34  35  36  38  39  43  44  45  46  47  48  49  50  53
  54  55  57  58  61  63  64  65  66  68  70  71  73  74  75  76  78  82
  84  85  86  87  90  91  92  93  94  95  96  99 100 101 102 105 106 108
 109 111 112 114 115 116 118 119 122 123 124 125 126 127 129 130] TEST: [  0   1   4  14  15  16  17  19  22  23  24  33  37  40  41  42  51  52
  56  59 

In [113]:
train_labels_dec[125]

2

In [0]:
num_epochs = 20
all_scores = []

In [0]:
from keras.utils.np_utils import to_categorical

In [127]:
for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])
  
  val_data = np.array([train_data_stand_pca[i] for i in val_index])
  val_targets = np.array([train_labels_dec[i] for i in val_index])

  one_hot_partial_train_targets = to_categorical(partial_train_targets)
  one_hot_val_targets = to_categorical(val_targets)

  model = build_model()
  model.fit(partial_train_data, one_hot_partial_train_targets, epochs = num_epochs, batch_size=1)

  val_loss, val_accuracy = model.evaluate(val_data, one_hot_val_targets)
  all_scores.append(val_accuracy)


Epoch 1/20
86/86 [==============================] - 2s 26ms/step - loss: 1.9407 - acc: 0.3953
Epoch 2/20
86/86 [==============================] - 0s 1ms/step - loss: 1.6031 - acc: 0.4070
Epoch 3/20
86/86 [==============================] - 0s 1ms/step - loss: 1.3895 - acc: 0.3721
Epoch 4/20
86/86 [==============================] - 0s 1ms/step - loss: 1.2570 - acc: 0.4302
Epoch 5/20
86/86 [==============================] - 0s 1ms/step - loss: 1.1663 - acc: 0.5000
Epoch 6/20
86/86 [==============================] - 0s 1ms/step - loss: 1.1149 - acc: 0.5233
Epoch 7/20
86/86 [==============================] - 0s 1ms/step - loss: 1.0783 - acc: 0.5581
Epoch 8/20
86/86 [==============================] - 0s 2ms/step - loss: 1.0488 - acc: 0.5698
Epoch 9/20
86/86 [==============================] - 0s 1ms/step - loss: 1.0192 - acc: 0.6047
Epoch 10/20
86/86 [==============================] - 0s 1ms/step - loss: 0.9985 - acc: 0.5930
Epoch 11/20
86/86 [==============================] - 0s 1ms/step - l

C'è un problema: keras.utils.to_categorical produces a one-hot encoded class vector, i.e. the multilabel-indicator mentioned in the error message. StratifiedKFold is not designed to work with such input; i.e. your y must be a 1-D array of your class labels.
Essentially, what you have to do is simply to invert the order of the operations: split first (using your intial y_train), and convert to_categorical afterwards.

In [128]:
all_scores

[0.46666667196485734,
 0.5348837230094644,
 0.5116279111352078,
 0.48888889418707954,
 0.5813953543818274,
 0.48837209648864216]

In [132]:
num_epochs = 30
all_accuracy_histories = []

for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])
  
  val_data = np.array([train_data_stand_pca[i] for i in val_index])
  val_targets = np.array([train_labels_dec[i] for i in val_index])

  one_hot_partial_train_targets = to_categorical(partial_train_targets)
  one_hot_val_targets = to_categorical(val_targets)

  model = build_model()
  history = model.fit(partial_train_data, one_hot_partial_train_targets, validation_data=(val_data, one_hot_val_targets), 
                      epochs= num_epochs, batch_size=1)
  
  accuracy_history = history.history
  all_accuracy_histories.append(accuracy_history)

Train on 86 samples, validate on 45 samples
Epoch 1/30
86/86 [==============================] - 3s 33ms/step - loss: 1.9002 - acc: 0.4070 - val_loss: 1.2925 - val_acc: 0.4222
Epoch 2/30
86/86 [==============================] - 0s 2ms/step - loss: 1.4549 - acc: 0.4186 - val_loss: 1.1442 - val_acc: 0.3778
Epoch 3/30
86/86 [==============================] - 0s 2ms/step - loss: 1.2224 - acc: 0.4419 - val_loss: 1.0904 - val_acc: 0.3778
Epoch 4/30
86/86 [==============================] - 0s 2ms/step - loss: 1.0745 - acc: 0.5349 - val_loss: 1.0658 - val_acc: 0.3556
Epoch 5/30
86/86 [==============================] - 0s 2ms/step - loss: 0.9930 - acc: 0.5814 - val_loss: 1.0755 - val_acc: 0.4444
Epoch 6/30
86/86 [==============================] - 0s 2ms/step - loss: 0.9479 - acc: 0.6395 - val_loss: 1.0949 - val_acc: 0.5111
Epoch 7/30
86/86 [==============================] - 0s 2ms/step - loss: 0.9192 - acc: 0.6628 - val_loss: 1.1144 - val_acc: 0.4667
Epoch 8/30
86/86 [===========================

In [0]:
history_dict=history.history

In [135]:
history_dict.keys()

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])